In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
from tensorflow.python.framework import ops
import random
import json

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
stemmer = LancasterStemmer() #stemmer to get stem of a word. ex. 'say' would be stem word of 'saying'.

In [3]:
with open('intents.json') as file:
    data = json.load(file)

print(type(data))
print(data)

<class 'dict'>
{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'how old is tim', 'what is your age', 'how old are you', 'age?'], 'responses': ['I am 18 years old!', '18 years young!'], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me Tim.', "I'm Tim!", "I'm Tim aka Tech With Tim."], 'context_set': ''}, {'tag': 'shop', 'patterns': ['Id like to buy something', 'whats on the menu', 'what do you reccommend?', 'could i get something to eat'], 'responses': ['We sell chocolate chip cookies for $2

In [4]:
# gives stemmed, tokenized words list from sentence pattern without words in ignore_words list
def clean_pattern(pattern, ignore_words):
    stemmed_pattern = []
    wrds = nltk.word_tokenize(pattern)
    for w in wrds:
        if w not in ignore_words:
            stemmed_pattern.append(stemmer.stem(w.lower()))
    return stemmed_pattern

stemmed_words = []
tags = []
ignore_words = ['!', '?', '.']
corpus = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        stemmed_pattern = clean_pattern(pattern, ignore_words)
        stemmed_words.extend(stemmed_pattern)
        corpus.append((stemmed_pattern, intent['tag']))
    if intent['tag'] not in tags:
        tags.append(intent['tag'])
        
# remove duplicates and sort
stemmed_words = sorted(list(set(stemmed_words)))
tags = sorted(list(set(tags)))

print(stemmed_words)
print(tags)
print(corpus)

['a', 'ag', 'am', 'anyon', 'ar', 'buy', 'cal', 'could', 'cya', 'day', 'do', 'eat', 'get', 'good', 'goodby', 'guy', 'hav', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'lat', 'leav', 'lik', 'menu', 'nam', 'of', 'old', 'on', 'op', 'reccommend', 'see', 'should', 'someth', 'the', 'ther', 'tim', 'to', 'up', 'what', 'when', 'yo', 'you']
['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']
[(['hi'], 'greeting'), (['how', 'ar', 'you'], 'greeting'), (['is', 'anyon', 'ther'], 'greeting'), (['hello'], 'greeting'), (['good', 'day'], 'greeting'), (['what', 'up'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you', 'lat'], 'goodbye'), (['goodby'], 'goodbye'), (['i', 'am', 'leav'], 'goodbye'), (['hav', 'a', 'good', 'day'], 'goodbye'), (['how', 'old'], 'age'), (['how', 'old', 'is', 'tim'], 'age'), (['what', 'is', 'yo', 'ag'], 'age'), (['how', 'old', 'ar', 'you'], 'age'), (['ag'], 'age'), (['what', 'is', 'yo', 'nam'], 'name'), (['what', 'should', 'i', 'cal', 'you'], 'name'), (['what', 'yo', 'nam'],

In [5]:
X = []
y = []

for item in corpus:
    bag = [] #array of 1 and 0. 1 if stemmed word is present stemmed pattern
    stemmed_pattern = item[0]
    for w in stemmed_words:
        if w in stemmed_pattern:
            bag.append(1)
        else:
            bag.append(0)
            
    tags_row = [] #array of 1 and 0. 1 for current tag and for everything else 0.
    current_tag = item[1]
    for tag in tags:
        if tag == current_tag:
            tags_row.append(1)
        else:
            tags_row.append(0)
    
    #for each item in corpus, X will be array indicating stemmed words and y array indicating tags
    X.append(bag)
    y.append(tags_row) 

X = np.array(X)
y = np.array(y)
print(X)
print(y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
[[0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]]


In [6]:
ops.reset_default_graph() #reset graph data

# neural network's layers
network = tflearn.input_data(shape= [None, len(X[0])]) #input layer
network = tflearn.fully_connected(network, 8) #1st hidden layer
network = tflearn.fully_connected(network, 8) #2nd hidden layer
network = tflearn.fully_connected(network, len(y[0]), activation= 'softmax') #output layer
network = tflearn.regression(network)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
# fitting model
model = tflearn.DNN(network, tensorboard_dir='tflearn_logs') #tensorboard_dir is path to store logs
model.fit(X, y, n_epoch=1000, batch_size=8, show_metric=True, shuffle= True) #n_epoch:no. of times model will see same data
model.save('chatbot_model.tflearn')

Training Step: 3999  | total loss: 0.08520 | time: 0.024s
| Adam | epoch: 1000 | loss: 0.08520 - acc: 0.9947 -- iter: 24/26
Training Step: 4000  | total loss: 0.07763 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.07763 - acc: 0.9952 -- iter: 26/26
--
INFO:tensorflow:F:\ChatBot\chatbot_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
